In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import re
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from unsloth import FastLanguageModel
import wandb
from datasets import load_dataset, Dataset, DatasetDict
from trl import SFTTrainer
from transformers import TrainingArguments

This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
feature_importance = {
    "Loan amount": 1.2,
    "DTI": 1.5,
    "Employment Title": 0.8,
    "Employment Length": 1.0,
    "Home Ownership": 1.1,
    "Annual Income": 1.6,
    "Verification Status": 1.0,
    "Grade": 2.0,
    "Purpose": 0.9,
    "Description": 0.7,
    "Title": 0.8,
    "Open Accounts": 1.3
}

In [3]:
loan_data_path = "example"
dataset = load_dataset(loan_data_path)

In [4]:
print(dataset["train"])
train_data = dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 167153
})


In [5]:
''' finetune on small balanced samples
# balance data sample
label_1_data = [data for data in train_data if data['label'] == 1]
label_0_data = [data for data in train_data if data['label'] == 0]

num_label_1 = len(label_1_data)
balanced_label_0_data = random.sample(label_0_data, num_label_1)
balanced_data = label_1_data + balanced_label_0_data

# random data layout
random.shuffle(balanced_data)

dataset = Dataset.from_list(balanced_data)

print(f"Original dataset size: {len(train_data)}")
print(f"Balanced dataset size: {len(balanced_data)}")
print(f"Number of label 1 samples: {len(label_1_data)}")
print(f"Number of label 0 samples: {len(balanced_label_0_data)}")
'''

' finetune on small balanced samples\n# balance data sample\nlabel_1_data = [data for data in train_data if data[\'label\'] == 1]\nlabel_0_data = [data for data in train_data if data[\'label\'] == 0]\n\nnum_label_1 = len(label_1_data)\nbalanced_label_0_data = random.sample(label_0_data, num_label_1)\nbalanced_data = label_1_data + balanced_label_0_data\n\n# random data layout\nrandom.shuffle(balanced_data)\n\ndataset = Dataset.from_list(balanced_data)\n\nprint(f"Original dataset size: {len(train_data)}")\nprint(f"Balanced dataset size: {len(balanced_data)}")\nprint(f"Number of label 1 samples: {len(label_1_data)}")\nprint(f"Number of label 0 samples: {len(balanced_label_0_data)}")\n'

In [6]:
train_data = [data for data in train_data]

print(len(train_data))
random.shuffle(train_data)
dataset = Dataset.from_list(train_data)

print(f"Original dataset size: {len(dataset)}")

167153
Original dataset size: 167153


In [7]:
def rename_columns(example):
    example["loan_data"] = example.pop("text")
    example["labels"] = example.pop("label")
    return example

dataset = dataset.map(rename_columns)
print(dataset)
print(f"dataset size: {len(dataset)}")
print(dataset[0])

Map:   0%|          | 0/167153 [00:00<?, ? examples/s]

Dataset({
    features: ['loan_data', 'labels'],
    num_rows: 167153
})
dataset size: 167153
{'loan_data': 'Loan amount: 22600, DTI: 38.8, Employment Title: STATION COMMANDER R&R NCO, Employment Length: 10+ years, Home Ownership: MORTGAGE, Annual Income: 60000.0, Verification Status: Source Verified, Grade: E-E4, Purpose: debt_consolidation, Description: nan, Title: Debt consolidation, Open Accounts: 11', 'labels': 0}


In [8]:
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # True to use 4bit quantization / reduce memory usage (for T4 GPU)

In [9]:
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/mistral-7b-v0.2-bnb-4bit",
    model_name = "model/Mistral-7B-Instruct-v0.3",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA A800-SXM4-80GB. Max memory: 79.325 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

model/Mistral-7B-Instruct-v0.3 does not have a padding token! Will use pad_token = [control_768].


In [10]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # can improve fine-tuning, at attention/feed fwd layers
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha = 16, # more change to pre-train weights but care overfitting
    lora_dropout = 0.05, # any, but 0 if perf opti.
    bias = "none",    # any, but "none" is perf  opti.
    use_gradient_checkpointing = True,
    random_state = 11,
    use_rslora = False,  # support rank stabilized LoRA
    loftq_config = None, # LoftQ support
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.8 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [11]:
# Prompt Preparation
prompt = """You are a highly intelligent and detailed artificial intelligence assistant with a deep understanding of financial data, specifically in predicting loan defaults.
Your task is to accurately classify loan data into one of two possible outcomes:
- 0: The loan is fully paid off (no default).
- 1: The loan has defaulted (borrower failed to meet the repayment terms).

The input data will provide various details about the loan and the borrower's financial situation. Your goal is to carefully analyze this information and determine the loan's status based on the provided features.

You are expected to generate a response that is one of the following labels:
- 0: The loan is fully paid off.
- 1: The loan has defaulted.

Your classification must be precise and match the best possible outcome for the given loan data.

Here is the loan data you need to classify:
### Loan Data:
{loan_data}
### Loan Status:
{loan_status}"""

In [12]:
# Add EOS special token, according to previously loaded tokenizer
EOS_TOKEN = tokenizer.eos_token
def format_prompts(examples):
    inputs = examples["loan_data"]
    outputs = examples["labels"]
    texts = []
    for inp, output in zip(inputs, outputs):
        # Add end of string token to prevent infinite generations.
        text = prompt.format(loan_data=inp, loan_status=output) + EOS_TOKEN
        texts.append(text)
    return {"text":texts}

In [13]:
# Building prompts
dataset= dataset.map(format_prompts, batched = True)

# Print a sample :
print("dataset:", dataset)
print(dataset[0]['text'])

Map:   0%|          | 0/167153 [00:00<?, ? examples/s]

dataset: Dataset({
    features: ['loan_data', 'labels', 'text'],
    num_rows: 167153
})
You are a highly intelligent and detailed artificial intelligence assistant with a deep understanding of financial data, specifically in predicting loan defaults.
Your task is to accurately classify loan data into one of two possible outcomes:
- 0: The loan is fully paid off (no default).
- 1: The loan has defaulted (borrower failed to meet the repayment terms).

The input data will provide various details about the loan and the borrower's financial situation. Your goal is to carefully analyze this information and determine the loan's status based on the provided features.

You are expected to generate a response that is one of the following labels:
- 0: The loan is fully paid off.
- 1: The loan has defaulted.

Your classification must be precise and match the best possible outcome for the given loan data.

Here is the loan data you need to classify:
### Loan Data:
Loan amount: 22600, DTI: 38.8, E

In [16]:
# Train the model
training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    num_train_epochs = 1,
    # max_steps = 110,
    learning_rate = 2e-4, # 2e-5
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 11,
    output_dir = "outputs/mistral-7b-instruct-v0.3-0921",
    run_name = "mistral-7b-instruct-v0.3",
    logging_strategy = 'steps',
    logging_steps = 1,
    save_strategy="steps",
    save_steps=10,
    save_total_limit = 2,
    report_to = "wandb",
    )

In [17]:
# init the trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # if packing = False, else default to None
    packing = False, # more speed if packing short sequences. Maybe later
    args = training_args
)
print("trainer.dataset:", trainer.train_dataset)
print("trainer.train_dataset[0]:", trainer.train_dataset[0])

Map (num_proc=2):   0%|          | 0/167153 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


trainer.dataset: Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 167153
})
trainer.train_dataset[0]: {'input_ids': [1, 1763, 1228, 1032, 7184, 19694, 1072, 11305, 19046, 11663, 14660, 1163, 1032, 4302, 7167, 1070, 6361, 1946, 29493, 10875, 1065, 7550, 1056, 10744, 21700, 29491, 781, 11927, 4406, 1117, 1066, 25097, 1643, 2343, 10744, 1946, 1546, 1392, 1070, 1757, 3340, 18782, 29515, 781, 29501, 29473, 29502, 29515, 1183, 10744, 1117, 6662, 6675, 1573, 1093, 2278, 3137, 1377, 781, 29501, 29473, 29508, 29515, 1183, 10744, 1427, 3137, 1054, 1093, 4390, 1439, 1031, 5447, 1066, 3415, 1040, 2326, 1107, 1234, 4239, 1377, 781, 781, 1782, 3555, 1946, 1390, 3852, 4886, 4930, 1452, 1040, 10744, 1072, 1040, 14987, 1031, 29510, 29481, 6361, 5388, 29491, 4372, 6309, 1117, 1066, 10315, 21533, 1224, 2639, 1072, 8423, 1040, 10744, 29510, 29481, 4193, 3586, 1124, 1040, 4625, 4958, 29491, 781, 781, 2744, 1228, 4117, 1066, 9038, 1032, 3667, 1137, 1117, 1392, 1070, 1040, 3064, 13267, 

In [18]:
'''train'''
trainer = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 167,153 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 20,894
 "-____-"     Number of trainable parameters = 41,943,040
wandb: Currently logged in as: junjie_chiang (jjchiang). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,2.088400
2,1.986200
3,1.863100
4,1.489700
5,1.177400
6,0.976500
7,0.737200
8,0.342400
9,0.346300
10,0.392600


KeyboardInterrupt: 